In [1]:
from scripts.experiments.symmetries import pauli_reduce, clifford_symmetry, get_coupled_qudits_by_gate
from sympleq.models.Ising import ising_chain_hamiltonian, ising_2d_hamiltonian, heuristic_clifford_symmetry
from sympleq.core.circuits.gate_decomposition_to_circuit import gate_to_circuit
import numpy as np

In [2]:

N = 4
J = 1
h = 0.5
H = ising_chain_hamiltonian(N, J, h, periodic=True)

# print(H)

h_red, conditioned_hamiltonians, C_F, all_phases = pauli_reduce(H)
# print(h_red)
# print(conditioned_hamiltonians)


In [3]:
H_c = H  #conditioned_hamiltonians[0]
F, S, T = clifford_symmetry(H_c, check_symmetry=True)

# print(F.symplectic)
# print(heuristic_clifford_symmetry(N))
assert np.all(F.symplectic == heuristic_clifford_symmetry(N).symplectic)
print(F.phase_vector)
print(heuristic_clifford_symmetry(N).phase_vector)
assert F == heuristic_clifford_symmetry(N)
C_F = gate_to_circuit(F)
# C_S = gate_to_circuit(S)
# C_T = gate_to_circuit(T)
assert C_F.act(H_c).to_standard_form() == H_c.to_standard_form()
# assert C_S.act(C_T.act(H_c)).standard_form() == H_c.standard_form()
U_F = C_F.unitary().toarray()
H_hilbert = H_c.to_hilbert_space().toarray()
assert np.all(np.abs(U_F @ H_hilbert @ U_F.conj().T - H_hilbert) < 1e-8)
print('Found unitary symmetry!!!')

[0 0 0 0 1 1 1 1]
[0 0 0 0 1 1 1 1]
Found unitary symmetry!!!


In [14]:
# print(S.symplectic)
# assert T.inv().act(S.act(T.act(H_c))).standard_form() == H_c.standard_form()
print(H_c.n_qudits())
print(get_coupled_qudits_by_gate(F))
print(get_coupled_qudits_by_gate(S))

6
[6.]
[2. 2. 2.]


In [5]:
n_x = 3
n_y = 2
J_zz = 1
h_x = 0.5
H_2d = ising_2d_hamiltonian(n_x, n_y, J_zz, h_x, periodic=False)

h_red, conditioned_hamiltonians, C_F, all_phases = pauli_reduce(H_2d)
print(conditioned_hamiltonians[0].n_qudits())
print(conditioned_hamiltonians[0].n_paulis())

5
13


In [4]:
F, S, T = clifford_symmetry(conditioned_hamiltonians[0])


Found symmetry


In [8]:
print(F.symplectic)

[[0 1 0 0 1 1 1 1]
 [1 0 0 0 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 1 0 1 1 1 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0]]


In [16]:
from sympleq.core.circuits import Gate
from scripts.experiments.symmetries.src.block_decomposition import block_decompose, ordered_block_sizes


F_tst = Gate('test', [0, 1], np.asarray([[1, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]), phase_vector=np.array([0, 0, 0, 0]), dimensions=[3, 3])

S, T = block_decompose(F_tst.symplectic, 3, min_block_size=4)
print(S)
print(T)

print(ordered_block_sizes(S, 3))
print(ordered_block_sizes(F_tst.symplectic, 3))



KeyboardInterrupt: 